# Data cleaning

### Imports
Import libraries and write settings here.

In [207]:
%load_ext autoreload
%autoreload
%load_ext dotenv
%dotenv

import pandas as pd
import os
import qgrid
import ipywidgets
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


### Set up extractions from each nrcan csv

In [208]:
#extract target name from table name
def substring_table(tableName):
    start = tableName.find(": ") + len(": ")
    end = tableName.find(" Secondary")
    substring = tableName[start:end]
    return substring

#rename Source and Value columns from table
def rename_columns(df):
    df.columns = ['Source', 'Value']
    return df

#Concatenate table_id number
def write_table_id(index):
    table_id = 'id_ca_e_' + str(index)
    return table_id

#add additional columns to table
def add_other_data(df, targetName, table_id):
    df.insert(1, 'Target', targetName)
    df.insert(3, 'Units', 'Petajoules')
    df.insert(4, 'Table ID', table_id)
    return df

#extract only relevant data
#note locations are hardcodes for nrcan files types -- all are equivalent
def extract_relevant_data(df):
    df = df.drop(df.iloc[:, 2:29], inplace = False, axis = 1)
    df = df.iloc[13:23,:]
    df = df.drop('Back to main menu',1)
    return df

def extract_data_last_two(df):
    df = df.drop(df.iloc[:, 2:29], inplace = False, axis = 1)
    df = df.iloc[13:23,:]
    df = df.drop(df.iloc[:, 3:6], inplace = False, axis = 1)
    df = df.drop('Back to main menu',1)
    return df

qgrid.enable()

### Loop Through Tables 9 - 67 and append to master df

In [209]:
filePath = 'data/idca2017e/id_ca_e_'
net_df = pd.DataFrame(columns=['Source','Target','Value','Units','Table ID'])

#index through all tables in idca2017e folder
for x in range(9,66):
    fileIndex = str(x)
    path = filePath + fileIndex + '.csv'
    df.iloc[0:0]
    df = pd.read_csv(path, encoding='cp1252', header=0)
    tableName = df.iloc[6,0]
    targetName = substring_table(tableName)
    table_id = write_table_id(fileIndex)
    df = extract_relevant_data(df)
    df = rename_columns(df)
    df = add_other_data(df, targetName, table_id)
    net_df = net_df.append(df)

In [210]:
#index through last two tables which randomly have 3 extra columns at the end
for x in range(66, 68):
    fileIndex = str(x)
    path = filePath + fileIndex + '.csv'
    df.iloc[0:0]
    df = pd.read_csv(path, encoding='cp1252', header=0)
    tableName = df.iloc[6,0]
    targetName = substring_table(tableName)
    table_id = write_table_id(fileIndex)
    df = extract_data_last_two(df)
    df = rename_columns(df)
    df = add_other_data(df, targetName, table_id)
    net_df = net_df.append(df)

display(net_df)


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [212]:
df = net_df 

### Drop zeroes and hidden / na values

In [216]:
# replace confidential labels and n/a with nan
df.replace("X", np.nan, inplace=True)
df.replace("n.a.", np.nan, inplace=True)
  
# Convert values to numeric
df['Value'] = pd.to_numeric(df['Value'])

display(df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

# Export data

In [219]:
#export data with zeroes
csvName = "id_ca_e_compiled.csv"
df.to_csv(csvName, index=False)

In [222]:
df_no_z = df[df.Value != 0]
display(df_no_z)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [224]:
#export data without zeroes
csvName2 = "id_ca_e_compiled_no_Zeroes.csv"
df_no_z.to_csv(csvName2, index=False)